
# Concept Drift Benchmark — Baseline (v2)
**ShapeDD (truyền thống)** + **DDM / Page‑Hinkley / ADWIN / MDDM / FHDDM / FHDDMS**  
**Streams:** SEA, Rotating Hyperplane, LED_abrupt, LED_gradual, Interchanging RBF

> Notebook này mở rộng bản baseline trước: bổ sung **streams synthetic** và **detector** phổ biến.



## Tiêu chí đánh giá (giải thích ngắn gọn)

**Nhóm phân loại (hiệu năng dự đoán):**
- **Accuracy (Prequential Accuracy):** tỷ lệ dự đoán đúng qua *toàn bộ dòng*, theo quy trình *predict-then-update*.  
  \( \text{Acc} = \frac{\sum_{t} \mathbb{1}(\hat{y}_t = y_t)}{T} \).
- **Precision / Recall / F1 (macro):** tính trung bình đều trên các lớp (tránh lệch khi mất cân bằng).
  - \( \text{Precision}_k = \frac{TP_k}{TP_k + FP_k} \), \( \text{Recall}_k = \frac{TP_k}{TP_k + FN_k} \).
  - \( F1_k = \frac{2\cdot \text{Prec}_k \cdot \text{Rec}_k}{\text{Prec}_k + \text{Rec}_k} \).
  - **Macro-F1** = trung bình của \(F1_k\) trên tất cả lớp.

**Nhóm drift detection (chất lượng phát hiện):**
- **#Detections:** tổng số báo động (alarms) tạo ra.
- **False Alarms (FA):** báo động không khớp bất kỳ drift thật nào (không nằm trong khoảng [drift, drift+tol]).
- **Detection Delay (mean):** trung bình độ trễ giữa mốc drift thật và báo động đầu tiên **sau** mốc đó (bỏ qua trường hợp không phát hiện).

> Lưu ý: Một detector tốt nên có **Acc cao**, **Macro-F1 cao**, **Delay thấp**, **FA thấp** (đổi lại có thể trade‑off).


In [21]:

import math, random, time
from dataclasses import dataclass
from typing import List, Tuple, Optional, Dict
import numpy as np
from collections import deque, defaultdict
import pandas as pd

np.random.seed(42)
random.seed(42)


## 1) Stream Generators

In [22]:

class SEAStream:
    """SEA concepts (binary) với drift đột ngột trên ngưỡng."""
    def __init__(self, length=10000, thresholds=(7, 8, 9, 9.5), drift_points=(2500, 5000, 7500), noise=0.0):
        self.length = length
        self.thresholds = thresholds
        self.drift_points = list(drift_points)
        self.noise = noise

    def generate(self):
        tps = [0] + self.drift_points + [self.length]
        def thr_for(i):
            for k in range(len(tps)-1):
                if tps[k] <= i < tps[k+1]:
                    return self.thresholds[k]
            return self.thresholds[-1]
        Xs = np.random.rand(self.length, 3)*10.0
        y = np.zeros(self.length, dtype=int)
        for i in range(self.length):
            thr = thr_for(i)
            label = 0 if (Xs[i,0] + Xs[i,1]) <= thr else 1
            if self.noise>0 and np.random.rand()<self.noise:
                label = 1-label
            y[i]=label
        return Xs, y, self.drift_points


In [23]:

class RotatingHyperplane:
    """Incremental drift: vector pháp tuyến quay dần dần (binary)."""
    def __init__(self, length=10000, d=10, angle_per_step=2*np.pi/20000, noise=0.0, abrupt_points=()):
        self.length=length; self.d=d
        self.angle_per_step=angle_per_step
        self.noise=noise
        self.abrupt_points = set(abrupt_points)

    def generate(self):
        X = np.random.randn(self.length, self.d)
        y = np.zeros(self.length, dtype=int)
        angle=0.0
        for i in range(self.length):
            angle += self.angle_per_step
            c, s = math.cos(angle), math.sin(angle)
            w = np.zeros(self.d); w[0]=c; w[1]=s
            if i in self.abrupt_points:
                w = -w
            score = X[i].dot(w)
            label = 1 if score>=0 else 0
            if self.noise>0 and np.random.rand()<self.noise:
                label = 1-label
            y[i]=label
        drift_points = [self.length//3, 2*self.length//3]
        return X, y, drift_points


In [24]:

def seven_segment_digit(bits7):
    # map 7 segments to a digit index by pattern (simplified; not unique)
    return int(sum(bits7) % 10)

class LEDStream:
    """LED generator: 24 binary attrs, 7 quan trọng. 
    - abrupt: tại các mốc drift -> hoán vị (permute) vị trí 7 bit quan trọng.
    - gradual: chuyển dần từ mapping cũ sang mới trong khoảng g_len.
    """
    def __init__(self, length=10000, mode='abrupt', drift_points=(3000, 6000, 8000), g_len=500, noise=0.05):
        self.length=length; self.mode=mode
        self.drift_points=list(drift_points); self.g_len=g_len; self.noise=noise

    def generate(self):
        d=24
        important = list(range(7))  # initial 7 important indices
        X = np.zeros((self.length, d), dtype=int)
        y = np.zeros(self.length, dtype=int)
        perm = list(range(d))
        next_perm = perm.copy()
        dp_idx=0
        def new_mapping(old_imp):
            # pick 7 new distinct indices uniformly
            cand = list(range(d))
            np.random.shuffle(cand)
            return sorted(cand[:7])
        pending = None  # for gradual

        for t in range(self.length):
            # feature generation
            X[t] = (np.random.rand(d)<0.5).astype(int)
            # gradually change mapping if needed
            if self.mode=='abrupt':
                if dp_idx < len(self.drift_points) and t==self.drift_points[dp_idx]:
                    important = new_mapping(important)
                    dp_idx+=1
            else:  # gradual
                if dp_idx < len(self.drift_points) and t==self.drift_points[dp_idx]:
                    pending = new_mapping(important)
                    start = t
                    dp_idx+=1
                if pending is not None:
                    alpha = min(1.0, (t - start)/max(1,self.g_len))
                    # probabilistically choose new mapping
                    if np.random.rand()<alpha:
                        important = pending
                        pending = None

            bits7 = X[t, important]
            lbl = seven_segment_digit(bits7)
            if self.noise>0 and np.random.rand()<self.noise:
                lbl = (lbl + np.random.randint(1,10))%10
            y[t]=lbl
        return X, y, self.drift_points


In [25]:

class InterchangingRBF:
    """RBF clusters, class labels of clusters hoán đổi tại các mốc drift (recurring/abrupt)."""
    def __init__(self, length=10000, d=10, n_centers=6, drift_points=(3000, 7000), noise=0.0):
        self.length=length; self.d=d; self.n_centers=n_centers
        self.drift_points=list(drift_points); self.noise=noise

    def generate(self):
        # init centers and class labels
        centers = np.random.randn(self.n_centers, self.d)*2.0
        labels = np.array([i%2 for i in range(self.n_centers)], dtype=int)  # binary classes
        X = np.zeros((self.length, self.d))
        y = np.zeros(self.length, dtype=int)
        dp_set = set(self.drift_points)
        for t in range(self.length):
            k = np.random.randint(0, self.n_centers)
            X[t] = centers[k] + 0.5*np.random.randn(self.d)
            lbl = labels[k]
            if self.noise>0 and np.random.rand()<self.noise:
                lbl = 1-lbl
            y[t]=lbl
            if t in dp_set:
                # swap class labels by rotating
                labels = 1 - labels  # simple invert all
        return X, y, self.drift_points


## 2) Online Learner: Gaussian Naive Bayes (incremental)

In [26]:

class OnlineGaussianNB:
    def __init__(self, n_features, n_classes=2, var_smoothing=1e-9):
        self.n_features = n_features
        self.n_classes = n_classes
        self.var_smoothing = var_smoothing
        self.counts = np.zeros(n_classes, dtype=float)
        self.means = np.zeros((n_classes, n_features), dtype=float)
        self.M2 = np.zeros((n_classes, n_features), dtype=float)
        self._eps = 1e-12

    def partial_fit(self, X, y):
        X = np.atleast_2d(X); y = np.atleast_1d(y)
        for xi, yi in zip(X, y):
            c = int(yi) if yi < self.n_classes else int(yi % self.n_classes)
            self.counts[c] += 1.0
            delta = xi - self.means[c]
            self.means[c] += delta / max(self.counts[c],1.0)
            delta2 = xi - self.means[c]
            self.M2[c] += delta * delta2

    def _vars(self):
        var = np.zeros_like(self.M2)
        for c in range(self.n_classes):
            denom = max(self.counts[c]-1.0, 1.0)
            var[c] = self.M2[c] / denom + self.var_smoothing
        return var

    def predict_proba(self, X):
        X = np.atleast_2d(X)
        var = self._vars()
        priors = (self.counts + self._eps)/(self.counts.sum() + self.n_classes*self._eps)
        logp = []
        for c in range(self.n_classes):
            # For each class c, compute log probability for all samples in X
            # var[c] and self.means[c] are 1D arrays with shape (n_features,)
            # X has shape (n_samples, n_features)
            log_var_term = -0.5 * np.sum(np.log(2*np.pi*var[c]))  # scalar
            diff_sq = ((X - self.means[c])**2) / var[c]  # shape (n_samples, n_features)
            quad_term = -0.5 * np.sum(diff_sq, axis=1)  # shape (n_samples,)
            lp = log_var_term + quad_term + np.log(priors[c]+self._eps)
            logp.append(lp)
        logp = np.vstack(logp).T
        m = np.max(logp, axis=1, keepdims=True)
        p = np.exp(logp - m); p = p/np.sum(p, axis=1, keepdims=True)
        return p

    def predict(self, X):
        return np.argmax(self.predict_proba(X), axis=1)


## 3) Drift Detectors

In [27]:

class ShapeDD:
    def __init__(self, w_ref=200, w_cur=200, calib_size=1000, q=0.995, min_delay=50):
        self.w_ref = w_ref; self.w_cur = w_cur
        self.calib_size = calib_size; self.q = q; self.min_delay = min_delay
        self.buffer = deque(maxlen=w_ref + w_cur + 5)
        self.t = 0; self.last_alarm_t = -10**9
        self.calib_stats = []; self.thr = None; self.alarms = []

    @staticmethod
    def _moments(X):
        X = np.asarray(X); 
        if X.ndim==1: X = X.reshape(-1,1)
        m = X.mean(axis=0); std = X.std(axis=0) + 1e-9
        z = (X - m)/std
        skew = np.mean(z**3, axis=0); kurt = np.mean(z**4, axis=0) - 3.0
        return m, std, skew, kurt

    def _shape(self, X):
        m, s, g, k = self._moments(X); return np.concatenate([m,s,g,k])

    def update(self, x):
        self.t += 1; self.buffer.append(x)
        if len(self.buffer) < (self.w_ref + self.w_cur): return None
        arr = np.array(self.buffer)
        ref = arr[-(self.w_ref + self.w_cur):-self.w_cur]; cur = arr[-self.w_cur:]
        s_ref = self._shape(ref); s_cur = self._shape(cur)
        scale = np.maximum(np.abs(s_ref), 1e-6)
        stat = np.linalg.norm((s_cur - s_ref)/scale)
        if self.t <= self.calib_size:
            self.calib_stats.append(stat); return None
        if self.thr is None and len(self.calib_stats)>10:
            self.thr = float(np.quantile(self.calib_stats, self.q))
        if self.thr is not None and stat>self.thr and (self.t - self.last_alarm_t)>=self.min_delay:
            self.last_alarm_t = self.t; self.alarms.append(self.t); return self.t
        return None

class DDM:
    def __init__(self, min_delay=50):
        self.n=0; self.p=0.0; self.pmin=float('inf'); self.smin=float('inf')
        self.alarms=[]; self.t=0; self.last_alarm_t=-10**9; self.min_delay=min_delay
    def update(self, is_error: bool):
        self.t+=1; self.n+=1; self.p += 1.0 if is_error else 0.0
        phat = self.p/self.n; s = math.sqrt(phat*(1-phat)/self.n)
        if phat + s < self.pmin + self.smin: self.pmin=phat; self.smin=s
        if phat + s >= self.pmin + 3*self.smin and (self.t - self.last_alarm_t)>=self.min_delay:
            self.alarms.append(self.t); self.last_alarm_t=self.t; return self.t
        return None

class PageHinkley:
    def __init__(self, delta=0.005, lambda_=5.0, min_delay=50):
        self.delta=delta; self.lambda_=lambda_
        self.t=0; self.mean=0.0; self.m_t=0.0; self.M_t=0.0
        self.alarms=[]; self.last_alarm_t=-10**9; self.min_delay=min_delay
    def update(self, value):
        self.t+=1
        self.mean = self.mean + (value - self.mean)/self.t
        self.m_t += (value - self.mean - self.delta)
        self.M_t = min(self.M_t, self.m_t)
        if (self.m_t - self.M_t) > self.lambda_ and (self.t - self.last_alarm_t)>=self.min_delay:
            self.alarms.append(self.t); self.m_t=0.0; self.M_t=0.0; self.last_alarm_t=self.t; return self.t
        return None

class ADWIN:
    """Simple ADWIN-like detector on binary error stream.
    Maintains a variable-length window and checks all cut points for mean change with Hoeffding-like bound.
    """
    def __init__(self, delta=0.002, min_window=50, min_delay=50):
        self.delta=delta; self.min_window=min_window
        self.win=deque(); self.alarms=[]; self.t=0; self.last_alarm_t=-10**9; self.min_delay=min_delay
    def update(self, value):
        self.t+=1; self.win.append(value)
        if len(self.win) < self.min_window: return None
        changed=False
        n=len(self.win); arr=np.array(self.win, dtype=float)
        mu = arr.mean()
        # check a few candidate cuts (not all for speed)
        for k in np.linspace(self.min_window//2, n-self.min_window//2, num=10, dtype=int):
            left = arr[:k]; right = arr[k:]
            mu1, mu2 = left.mean(), right.mean()
            eps = math.sqrt( (1/(2*k)) * math.log(4/self.delta) ) + math.sqrt( (1/(2*(n-k))) * math.log(4/self.delta) )
            if abs(mu1 - mu2) > eps:
                changed=True; break
        if changed and (self.t - self.last_alarm_t)>=self.min_delay:
            # shrink window by dropping older half
            for _ in range(len(self.win)//2): self.win.popleft()
            self.alarms.append(self.t); self.last_alarm_t=self.t; return self.t
        return None

class MDDM:
    """McDiarmid Drift Detection (simplified): compare weighted averages in two halves of a sliding window."""
    def __init__(self, W=400, delta=0.002, min_delay=50):
        self.W=W; self.delta=delta; self.buf=deque(maxlen=W)
        self.alarms=[]; self.t=0; self.last_alarm_t=-10**9; self.min_delay=min_delay
    def update(self, value):
        self.t+=1; self.buf.append(float(value))
        if len(self.buf)<self.W: return None
        arr=np.array(self.buf); k=self.W//2
        left, right = arr[:k], arr[k:]
        # weights increasing (MDDM-A style)
        w = np.arange(1, k+1, dtype=float); w/=w.sum()
        mu1 = np.sum(left * w); mu2 = np.sum(right * w)
        # McDiarmid bound with weights: sum c_i^2 where c_i are weights' bounds in [0,1]
        Ci2 = np.sum((w)**2)
        eps = math.sqrt(0.5*Ci2*math.log(2.0/self.delta))
        if (mu2 - mu1) > eps and (self.t - self.last_alarm_t)>=self.min_delay:
            self.alarms.append(self.t); self.last_alarm_t=self.t; return self.t
        return None

class FHDDM:
    """Hoeffding Drift Detection on sliding window of correctness (1-correct,0-wrong or vice versa)."""
    def __init__(self, W=500, delta=0.002, min_delay=50):
        self.W=W; self.delta=delta; self.buf=deque(maxlen=W)
        self.mu_max=0.0; self.alarms=[]; self.t=0; self.last_alarm_t=-10**9; self.min_delay=min_delay
    def update(self, correct01):
        self.t+=1; self.buf.append(float(correct01))
        if len(self.buf)<self.W: return None
        mu = np.mean(self.buf)
        self.mu_max = max(self.mu_max, mu)
        eps = math.sqrt((1/(2*self.W))*math.log(1/self.delta))
        if (self.mu_max - mu) >= eps and (self.t - self.last_alarm_t)>=self.min_delay:
            self.alarms.append(self.t); self.mu_max = mu  # reset baseline
            self.last_alarm_t=self.t; return self.t
        return None

class FHDDMS:
    """Stacking FHDDM: short & long windows; alarm if any triggers."""
    def __init__(self, W_short=100, W_long=500, delta=0.002, min_delay=50):
        self.short = FHDDM(W=W_short, delta=delta, min_delay=min_delay)
        self.long = FHDDM(W=W_long,  delta=delta, min_delay=min_delay)
        self.alarms=[]; self.t=0
    def update(self, correct01):
        self.t+=1
        a1 = self.short.update(correct01)
        a2 = self.long.update(correct01)
        fired = False
        for a in (a1,a2):
            if a is not None: fired=True
        if fired:
            self.alarms.append(self.t)
            return self.t
        return None


## 4) Evaluation (prequential)

In [28]:

@dataclass
class RunResult:
    stream: str
    name: str
    accuracy: float
    macro_f1: float
    n_det: int
    false_alarms: int
    mean_delay: Optional[float]
    delays: List[int]
    alarms: List[int]
    drift_points: List[int]
    runtime_s: float

def compute_delays(alarms, drift_points, tol=200):
    drift_points = list(drift_points); alarms = sorted(alarms); used=set(); delays=[]
    for dp in drift_points:
        cand = [a for a in alarms if a>=dp and (a-dp)<=tol and a not in used]
        if cand:
            a=cand[0]; used.add(a); delays.append(a-dp)
        else:
            delays.append(np.nan)
    good=set()
    for dp in drift_points:
        good.update([a for a in alarms if a>=dp and (a-dp)<=tol])
    false_alarms = len([a for a in alarms if a not in good])
    md = np.nanmean(delays) if len(delays)>0 else np.nan
    return delays, false_alarms, md

def update_confusion(cm, y_true, y_pred, n_classes):
    cm[y_true, y_pred] += 1
    return cm

def metrics_from_cm(cm):
    # Accuracy
    acc = np.trace(cm)/np.sum(cm) if cm.sum()>0 else 0.0
    # Macro F1
    K = cm.shape[0]
    f1s=[]
    for k in range(K):
        TP = cm[k,k]
        FP = cm[:,k].sum() - TP
        FN = cm[k,:].sum() - TP
        prec = TP/(TP+FP) if (TP+FP)>0 else 0.0
        rec  = TP/(TP+FN) if (TP+FN)>0 else 0.0
        f1 = 2*prec*rec/(prec+rec) if (prec+rec)>0 else 0.0
        f1s.append(f1)
    macro_f1 = float(np.mean(f1s)) if len(f1s)>0 else 0.0
    return acc, macro_f1

def run_stream_experiment(stream_name, X, y, drift_points, detectors, learner):
    t0 = time.time()
    n = len(y)
    n_classes = int(np.max(y))+1
    cm = np.zeros((n_classes, n_classes), dtype=int)
    for i in range(n):
        xi = X[i]; yi = y[i]
        yhat = learner.predict(xi)[0] if learner.counts.sum()>0 else random.randint(0, n_classes-1)
        cm = update_confusion(cm, yi, yhat, n_classes)
        err = 0 if yhat==yi else 1
        # feed detectors
        for name, det in detectors.items():
            if isinstance(det, ShapeDD):
                det.update(xi)
            elif isinstance(det, (DDM, ADWIN, MDDM, FHDDM, FHDDMS, PageHinkley)):
                # For FHDDM(S), use correctness (1=correct)
                if isinstance(det, (FHDDM, FHDDMS)):
                    det.update(1-err==1)  # correct01
                else:
                    det.update(err)
        learner.partial_fit(xi, yi)
    # summarize per detector
    acc, macro_f1 = metrics_from_cm(cm)
    results=[]
    for name, det in detectors.items():
        alarms = getattr(det, 'alarms', [])
        delays, fa, md = compute_delays(alarms, drift_points)
        results.append(RunResult(
            stream=stream_name, name=name, accuracy=acc, macro_f1=macro_f1,
            n_det=len(alarms), false_alarms=fa, mean_delay=md, delays=delays,
            alarms=alarms, drift_points=drift_points, runtime_s=time.time()-t0
        ))
    return results


## 5) Chạy thử nghiệm (5 streams)

In [29]:

N = 15000
streams = []

# SEA abrupt
sea = SEAStream(length=N, thresholds=(7.0, 8.0, 9.0, 9.5), drift_points=(4000, 8000, 12000), noise=0.01)
streams.append(("SEA",) + sea.generate())

# Rotating hyperplane
rot = RotatingHyperplane(length=N, d=10, angle_per_step=2*np.pi/(N//2), noise=0.01)
streams.append(("RotatingHyperplane",) + rot.generate())

# LED abrupt (10 classes)
led_a = LEDStream(length=N, mode='abrupt', drift_points=(3000, 7000, 12000), g_len=0, noise=0.02)
streams.append(("LED_abrupt",) + led_a.generate())

# LED gradual
led_g = LEDStream(length=N, mode='gradual', drift_points=(4000, 9000), g_len=800, noise=0.02)
streams.append(("LED_gradual",) + led_g.generate())

# Interchanging RBF
irbf = InterchangingRBF(length=N, d=10, n_centers=6, drift_points=(5000, 10000), noise=0.01)
streams.append(("InterchangingRBF",) + irbf.generate())

all_rows=[]; all_details={}

for sname, Xs, ys, dps in streams:
    print(f"== Running on {sname} ==")
    n_classes = int(np.max(ys))+1
    dets = {
        "ShapeDD": ShapeDD(w_ref=200, w_cur=200, calib_size=1000, q=0.995, min_delay=100),
        "DDM": DDM(min_delay=100),
        "PageHinkley": PageHinkley(delta=0.001, lambda_=10.0, min_delay=100),
        "ADWIN": ADWIN(delta=0.002, min_window=80, min_delay=100),
        "MDDM": MDDM(W=400, delta=0.005, min_delay=100),
        "FHDDM": FHDDM(W=500, delta=0.002, min_delay=100),
        "FHDDMS": FHDDMS(W_short=100, W_long=500, delta=0.002, min_delay=100),
    }
    learner = OnlineGaussianNB(n_features=Xs.shape[1], n_classes=n_classes)
    res = run_stream_experiment(sname, Xs, ys, dps, dets, learner)
    for r in res:
        all_rows.append({
            "stream": r.stream,
            "detector": r.name,
            "accuracy": r.accuracy,
            "macro_f1": r.macro_f1,
            "n_detections": r.n_det,
            "false_alarms": r.false_alarms,
            "mean_delay": r.mean_delay,
            "runtime_s": r.runtime_s
        })
        all_details[(sname, r.name)] = r

df = pd.DataFrame(all_rows)
df


== Running on SEA ==


/tmp/ipykernel_59695/3804475212.py:27: RuntimeWarning: Mean of empty slice
  md = np.nanmean(delays) if len(delays)>0 else np.nan


== Running on RotatingHyperplane ==
== Running on LED_abrupt ==
== Running on LED_gradual ==
== Running on InterchangingRBF ==


,stream,detector,accuracy,macro_f1,n_detections,false_alarms,mean_delay,runtime_s
0,SEA,ShapeDD,0.887133,0.865932,44,42,99.000000,10.596353
1,SEA,DDM,0.887133,0.865932,108,104,26.000000,10.596593
2,SEA,PageHinkley,0.887133,0.865932,47,44,106.500000,10.596682
3,SEA,ADWIN,0.887133,0.865932,2,2,NaN,10.597302
4,SEA,MDDM,0.887133,0.865932,0,0,NaN,10.597501
5,SEA,FHDDM,0.887133,0.865932,2,2,NaN,10.597608
6,SEA,FHDDMS,0.887133,0.865932,4,3,110.000000,10.597673
7,RotatingHyperplane,ShapeDD,0.516733,0.516701,50,49,196.000000,10.447450
8,RotatingHyperplane,DDM,0.516733,0.516701,143,139,26.000000,10.447497
9,RotatingHyperplane,PageHinkley,0.516733,0.516701,104,102,56.000000,10.447533


In [30]:

out_csv = "/mnt/data/baseline_v2_results_summary.csv"
df.to_csv(out_csv, index=False)
print("Saved:", out_csv)


OSError: Cannot save file into a non-existent directory: '/mnt/data'

### (Tuỳ chọn) Vẽ timeline drift vs. alarm

In [ ]:

import matplotlib.pyplot as plt

def plot_timeline(r, n_points: int):
    plt.figure(figsize=(10,2))
    for dp in r.drift_points:
        plt.axvline(dp, linestyle="--", alpha=0.6)
    for a in r.alarms:
        plt.axvline(a, color="r", alpha=0.7)
    plt.xlim(0, n_points)
    plt.title(f"{r.stream} / {r.name}: drift (--) vs alarms (red)")
    plt.xlabel("time")
    plt.show()

# Ví dụ
plot_timeline(all_details[("SEA","ADWIN")], N)
plot_timeline(all_details[("LED_abrupt","FHDDMS")], N)
plot_timeline(all_details[("InterchangingRBF","ShapeDD")], N)
